In [1]:
# imports
import numpy as np
import pandas as pd
import sqlite3 as sq3
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
#importing the bike station data from the previous section
bike_data = pd.read_csv('bike_data.csv')
longitude = bike_data['longitude']
latitude = bike_data['latitude']

In [3]:
#Setting up API key
foursquare_key = os.environ['FOURSQUARE_API_KEY']

In [4]:
#Setting up foursqaure test url
url = 'https://api.foursquare.com/v3/places/search?ll=' + str(latitude[0]) + ',' + str(longitude[0]) +'&radius=1000'
print(url)
headers = {"Accept":"application/json"}
headers['Authorization'] = foursquare_key
result = requests.get(url,headers = headers)

https://api.foursquare.com/v3/places/search?ll=46.8294326924675,-71.24406570853023&radius=1000


In [5]:
#checking the test data to see if the relevant information is present
foursquare_data = result.json()
foursquare_data['results'][0]['categories'][0]['name']

'Grocery Store / Supermarket'

In [6]:
fsquare_list = []
for i in range(len(latitude)):
    url = 'https://api.foursquare.com/v3/places/search?ll=' + str(latitude[i]) + ',' + str(longitude[i]) +'&radius=1000'
    headers = {"Accept":"application/json"}
    headers['Authorization'] = foursquare_key
    result_test = requests.get(url, headers=headers)
    fsquare_list.append(result_test.json())

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
#function to filter restaurants in foursquare search
def restaurant_filter(df):
    restaurant = []
    for i in range(len(df['results'])):
        for n in range(len(df['results'][i]['categories'])):
            if 'Restaurant' in df['results'][i]['categories'][n]['name']:
                restaurant.append(df['results'][i])
                break
    return restaurant

In [9]:
restaurant_data =[]
for i in range(len(fsquare_list)):
    restaurant_data.append(restaurant_filter(fsquare_list[i]))

In [14]:
#Creating a dataframe
fs_food_data = pd.DataFrame(columns=['name','country','locality','near_station'])
fs_food_data

,name,country,locality,near_station


In [15]:
#function to extract data from the previously filtered list into a Dataframe
def data_extract(df, food_data, bike_data):
    for i in range(len(food_data)):
        for n in range(len(food_data[i])):
            name = food_data[i][n]['name']
            country = food_data[i][n]['location']['country']
            locality = food_data[i][n]['location']['locality']
            near_station = bike_data['name'][i]
            info = pd.Series({'name':name, 'country':country, 'locality':locality, 'near_station':near_station})
            print(info)
            df = pd.concat([df,info.to_frame().T],ignore_index=True)
    
    return df

Put your parsed results into a DataFrame

In [ ]:
#applying the extraction function
fs_food_data = data_extract(fs_food_data, restaurant_data, bike_data)

In [119]:
#exporting the resulting dataframe to a csv file
fs_food_data.to_csv('foursquare_food.csv', index=False)

# Yelp

In [21]:
#Setting up Yelp API key
yelp_key = os.environ['yelp_api_key']

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [78]:
#testing url response
url_yelp = 'https://api.yelp.com/v3/businesses/search?categories=restaurants&latitude=46.829433&longitude=-71.244066&radius=1000'
headers = {"Accept":"application/json"}
headers['Authorization'] = 'Bearer ' + yelp_key
result_yelp = requests.get(url_yelp ,headers = headers)
result_yelp

<Response [200]>

In [22]:
yelp_list =[]

#Getting the data for restaurant near each stations
for i in range(len(latitude)):
    url_yelp = 'https://api.yelp.com/v3/businesses/search?categories=restaurants&latitude=' + str(latitude[i]) + '&longitude=' + str(longitude[i]) +'&radius=1000'
    headers = {"Accept":"application/json"}
    headers['Authorization'] = 'Bearer ' + yelp_key
    result_yelp = requests.get(url_yelp, headers=headers)
    yelp_list.append(result_yelp.json())

In [23]:
#a detailed look at an example element of the yelp list
yelp_list[0]['businesses'][0]

{'id': 'pKRPuTHpUISDKgXUyAMYIQ',
 'alias': 'casa-calzone-québec-5',
 'name': 'Casa Calzone',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/q2lNYNUDOmkhfleYdooxnw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/casa-calzone-qu%C3%A9bec-5?adjust_creative=O6St2lkB-rsXWIoV6gzvTQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=O6St2lkB-rsXWIoV6gzvTQ',
 'review_count': 24,
 'categories': [{'alias': 'italian', 'title': 'Italian'}],
 'rating': 5.0,
 'coordinates': {'latitude': 46.8223786, 'longitude': -71.2415612},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '1298 Rue de la Pointe-aux-Lièvres',
  'address2': '',
  'address3': '',
  'city': 'Quebec City',
  'zip_code': 'G1L 4L9',
  'country': 'CA',
  'state': 'QC',
  'display_address': ['1298 Rue de la Pointe-aux-Lièvres',
   'Quebec City, QC G1L 4L9',
   'Canada']},
 'phone': '+14185223000',
 'display_phone': '+1 418-522-3000',
 'distance': 807.1911234743243}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [24]:
#Creatin the yelp dataframe
yelp_food_data = pd.DataFrame(columns=['name','country','city','review_count','rating','near_station'])
yelp_food_data

,name,country,city,review_count,rating,near_station


In [25]:
#function to extract information from the Yelp search
def data_extract_yelp(df, food_data, bike_data):
    for i in range(len(food_data)):
        for n in range(len(food_data[i]['businesses'])):
            name = food_data[i]['businesses'][n]['name']
            country = food_data[i]['businesses'][n]['location']['country']
            city = food_data[i]['businesses'][n]['location']['city']
            review_count = food_data[i]['businesses'][n]['review_count']
            rating = food_data[i]['businesses'][n]['rating']
            near_station = bike_data['name'][i]
            info = pd.Series({'name':name, 
                              'country':country, 
                              'city':city, 
                              'review_count':review_count,
                              'rating':rating,
                              'near_station':near_station})
            df = pd.concat([df,info.to_frame().T],ignore_index=True)
    return df

Put your parsed results into a DataFrame

In [26]:
#extracting the data into the yelp dataframe
yelp_food_data = data_extract_yelp(yelp_food_data, yelp_list, bike_data)

In [27]:
#the yelp dataframe
yelp_food_data

,name,country,city,review_count,rating,near_station
0,Casa Calzone,CA,Quebec City,24,5.0,Grand Marché
1,Chez Carlos Café,CA,Quebec City,6,5.0,Grand Marché
2,Resto la Mama Grilled Cheese,CA,Québec City,5,4.5,Grand Marché
3,Sous-Marins le Marinier,CA,Quebec City,3,5.0,Grand Marché
4,Nguyen,CA,Quebec City,2,5.0,Grand Marché
...,...,...,...,...,...,...
737,La Boîte à Pain,CA,Quebec City,101,4.0,Parc John-Munn
738,L'ENTRECOTE SAINT-JEAN,CA,Quebec City,160,4.0,Parc John-Munn
739,La Pizzaio,CA,Quebec City,37,5.0,Parc John-Munn
740,Casse-Crêpe Breton,CA,Quebec City,325,3.5,Parc John-Munn


In [120]:
#Exporting the data into csv file
yelp_food_data.to_csv('yelp_food.csv', index = False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

It is not possible to compare rating in this senario because the foursquare API did not provide any rating information for the establishments surveyed. However, it can be noted that the Yelp search resulted in 742 results while the foursquare search only provided 146 results. Also, the Yelp search provided much more information such as review_count, ratings, price and such. It can be concluded that Yelp yielded the better data in this senario. 